In [1]:
#@title connect to drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title install

!git clone https://github.com/HydeYYHH/dfl-colab /content/D

!pip install -q condacolab

import condacolab
condacolab.install()
!conda create -n dfl python=3.6

%cd /content/D
!source activate dfl;pip install -r requirements-colab.txt;pip list
!conda install cudatoolkit
!sudo ln -s /usr/local/cuda-12/targets/x86_64-linux/lib/libcusolver.so.11 /usr/local/cuda-12/targets/x86_64-linux/lib/libcusolver.so.10
# download pretrained model and faces
!mkdir /content/pretrained_model
!mkdir /content/pretrained_faces
!wget -P /content/pretrained_faces https://github.com/HydeYYHH/dfl-colab/releases/download/pretrain_faces/faceset.pak
!wget -P /content/pretrained_model https://github.com/HydeYYHH/dfl-colab/releases/download/untagged-c5785d342c687fee8b3c/decoder_dst.npy
!wget -P /content/pretrained_model https://github.com/HydeYYHH/dfl-colab/releases/download/untagged-c5785d342c687fee8b3c/decoder_src.npy
!wget -P /content/pretrained_model https://github.com/HydeYYHH/dfl-colab/releases/download/untagged-c5785d342c687fee8b3c/encoder.npy

In [12]:
#@title config
name = "haoyunlianlian" #@param {"type":"string"}
model = "Quick96" #@param ["SAEHD","Quick96"]
src = "" # @param {"type":"string"}
des = "/content/drive/MyDrive/shared/face_material/target/2.mp4" # @param {"type":"string"}
work_dir = "/content/drive/MyDrive/shared/DFLab" # @param {"type":"string"}
import os
if not os.path.exists(work_dir):
  if input("work_dir doesn't exist, do you want to create it?(y/n)").lower() == 'y':
    raise FileNotFoundError
work_dir = f"{work_dir}/{name}"
src_dir = f"{work_dir}/src"
des_dir = f"{work_dir}/des"
model_dir = f"{work_dir}/model"
!mkdir -p $src_dir
!mkdir -p $des_dir
!mkdir -p $model_dir


In [ ]:
#@title extract face
%cd /content/D

target = "des" #@param ["src","des"]
if (target == "src"):
  input_file = src
  input_dir = src_dir
else:
  input_file = des
  input_dir = des_dir

cmd = f"main.py videoed extract-video --input-file {input_file} --output-dir {input_dir}"
!source activate dfl;python $cmd
cmd = f"""
source activate dfl
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-12/targets/x86_64-linux/lib
main.py extract --input-dir {input_dir} --output-dir {input_dir}/aligned --detector s3fd
"""
!$cmd
cmd = f"main.py util --input-dir {input_dir}/aligned  --pack-faceset"
!source activate dfl;python $cmd


In [ ]:
#@title import from .zip
src = "/content/drive/MyDrive/DFLab/yuanzhang/src/00004.zip" #@param {"type":"string"}
des = "/content/drive/MyDrive/DFLab/yuanzhang/des/00010.zip" #@param {"type":"string"}

if src:
  !unzip $src -d $src_dir
if des:
  !unzip $des -d $des_dir

In [ ]:
#@title train
%cd /content/D

cmd = f"""
source activate dfl
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-12/targets/x86_64-linux/lib
python main.py train --training-data-src-dir {src_dir}/aligned --training-data-dst-dir {des_dir}/aligned --model-dir {model_dir} --model {model} --pretraining-data-dir /content/pretrained_faces"""
if model == "Quick96":
  cmd += " --pretrained-model-dir /content/pretrained_model"
!$cmd

/content/D
Running trainer.

Choose one of saved models, or enter a name to create a new model.
[r] : rename
[d] : delete

[0] : haoyunlianlian - latest
 : 0
0
Loading haoyunlianlian_Quick96 model...

Choose one or several GPU idxs (separated by comma).

[CPU] : CPU
  [0] : Tesla T4

[0] Which GPU indexes to choose? : 
0


Initializing models: 100% 5/5 [00:02<00:00,  2.46it/s]
Loaded 1055 packed faces from /content/drive/MyDrive/shared/DFLab/haoyunlianlian/src/aligned
Loaded 805 packed faces from /content/drive/MyDrive/shared/DFLab/haoyunlianlian/des/aligned
================ Model Summary ================
==                                           ==
==        Model name: haoyunlianlian_Quick96 ==
==                                           ==
== Current iteration: 57071                  ==
==                                           ==
==-------------- Model Options --------------==
==                                           ==
==        batch_size: 4                      ==
== 

In [ ]:
# @title convert to output
%cd /content/D

cmd = f"main.py merge --output-mask-dir {des_dir}/merged_mask --input-dir {des_dir} --output-dir {des_dir}/merged --aligned-dir {des_dir}/aligned --model-dir {model_dir} --model {model}"
!source activate dfl;python $cmd

cmd = f"main.py videoed video-from-sequence --input-dir {des_dir}/merged --output-file {work_dir}/result.mp4 --reference-file {des} --include-audio"
!source activate dfl;python $cmd